SQL Database Natural Language Query Interface
Query a real database (e.g., SQLite or PostgreSQL) using plain English.

Groq API key

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
groq = os.getenv("GROQ_API_KEY")


creating the database

In [7]:
import sqlite3

conn = sqlite3.connect("../example.db")
cursor = conn.cursor()


cursor.execute("DROP TABLE IF EXISTS employees")
cursor.execute("DROP TABLE IF EXISTS departments")

cursor.execute("""
CREATE TABLE departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT
)
""")

cursor.executemany("""
INSERT INTO departments (dept_id, dept_name)
VALUES (?, ?)
""", [
    (1, "Engineering"),
    (2, "Design"),
    (3, "Management"),
])


cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    role TEXT,
    salary INTEGER,
    dept_id INTEGER,
    FOREIGN KEY(dept_id) REFERENCES departments(dept_id)
)
""")



cursor.executemany("""
INSERT INTO employees (id, name, role, salary, dept_id)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, "Alice", "Engineer", 120000, 1),
    (2, "Bob", "Designer", 90000, 2),
    (3, "Charlie", "Manager", 150000, 3),
    (4, "David", "Engineer", 110000, 1),
    (5, "Eva", "Designer", 85000, 2),
])

conn.commit()
conn.close()


Importing the LLM

In [8]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192"
)

Connecting the database to LLM

In [9]:
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

db = SQLDatabase.from_uri("sqlite:///example.db")


toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
)


Testing

In [10]:
response = agent_executor.invoke("Who earns more than 100000?")
print(response)

response = agent_executor.invoke("Show all employees who work in the Design department and earn more than 80000.")
print(response['output'])

response = agent_executor.invoke("What is the average salary?")
print(response)

print(response['output'])


{'input': 'Who earns more than 100000?', 'output': 'Alice and Charlie earn more than 100000.'}
The employees who work in the Design department and earn more than 80000 are Bob with a salary of 90000 and Eva with a salary of 85000.


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I now know the final answer: The average salary is 111000.0.`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 